In [2]:
import csv
import mysql.connector
from mysql.connector import Error
import os
import chardet

def detect_encoding(file_path):
    """检测文件编码"""
    with open(file_path, 'rb') as f:
        raw_data = f.read(1024)
        result = chardet.detect(raw_data)
        return result.get('encoding', 'utf-8')

def import_txt_to_securities_info(txt_file_path, host, database, user, password):
    try:
        # 检测文件编码
        encoding = detect_encoding(txt_file_path)
        print(f"检测到文件编码: {encoding}")

        # 连接到MySQL数据库
        connection = mysql.connector.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )

        if connection.is_connected():
            cursor = connection.cursor()

            # 首先查询数据库中已有的数据
            print("查询数据库中已有数据...")
            cursor.execute("SELECT security_code, stock_discount_rate FROM securities_info")
            existing_data = {}
            for row in cursor.fetchall():
                # 将 Decimal 转换为 float 以便比较
                discount_rate = float(row[1]) if row[1] is not None else None
                existing_data[row[0]] = discount_rate
            print(f"数据库中已有 {len(existing_data)} 条记录")

            # 读取TXT文件并收集所有证券代码
            txt_security_codes = set()
            processed_rows = []

            with open(txt_file_path, 'r', encoding=encoding) as file:
                # 使用csv.reader读取制表符分隔的文件
                txt_data = csv.reader(file, delimiter='\t')

                for row_num, row in enumerate(txt_data, 1):
                    # 跳过空行
                    if not row or all(cell.strip() == '' for cell in row):
                        print(f"跳过第 {row_num} 行空行")
                        continue

                    # 确保每行有9个字段，不足的用None填充
                    while len(row) < 9:
                        row.append(None)

                    # 处理空值
                    processed_row = []
                    for i, value in enumerate(row):
                        if value == '' or value is None:
                            processed_row.append(None)
                        else:
                            # 对于数值字段，尝试转换为浮点数
                            if i in [2, 3, 4]:  # 第3、4、5个字段是数值
                                try:
                                    processed_row.append(float(value))
                                except ValueError:
                                    processed_row.append(None)
                            else:
                                processed_row.append(value.strip())

                    # 收集证券代码
                    security_code = processed_row[0]
                    if security_code:
                        txt_security_codes.add(security_code)
                        processed_rows.append(processed_row)

            # 找出需要删除的记录（在数据库中存在但在TXT中不存在）
            db_security_codes = set(existing_data.keys())
            codes_to_delete = db_security_codes - txt_security_codes

            # 准备插入和更新语句
            insert_query = """INSERT INTO securities_info (
                security_code, security_name, stock_discount_rate, financing_margin_ratio,
                securities_margin_ratio, financing_status, securities_status, remark, exchange_name
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""

            update_query = """UPDATE securities_info SET
                security_name = %s, stock_discount_rate = %s, financing_margin_ratio = %s,
                securities_margin_ratio = %s, financing_status = %s, securities_status = %s,
                remark = %s, exchange_name = %s
                WHERE security_code = %s"""

            # 删除查询
            delete_query = "DELETE FROM securities_info WHERE security_code = %s"

            # 处理数据
            insert_count = 0
            update_count = 0
            skip_count = 0
            delete_count = 0

            # 处理插入和更新
            for processed_row in processed_rows:
                # 提取security_code和stock_discount_rate
                security_code = processed_row[0]
                stock_discount_rate = processed_row[2]

                # 检查数据是否已存在
                if security_code in existing_data:
                    # 检查stock_discount_rate是否一致
                    db_discount_rate = existing_data[security_code]

                    # 处理可能的None值
                    if db_discount_rate is None and stock_discount_rate is None:
                        # 两者都为None，视为相同
                        print(f"证券代码 {security_code} 已存在且数据相同，跳过")
                        skip_count += 1
                        continue
                    elif (db_discount_rate is None and stock_discount_rate is not None) or \
                         (db_discount_rate is not None and stock_discount_rate is None):
                        # 一个为None，另一个不为None，需要更新
                        need_update = True
                    else:
                        # 两者都不为None，比较数值
                        # 使用很小的阈值比较浮点数，避免精度问题
                        if abs(db_discount_rate - stock_discount_rate) > 0.001:
                            need_update = True
                        else:
                            need_update = False

                    if need_update:
                        # 数据不一致，执行更新
                        # 准备更新参数：前8个字段 + security_code作为WHERE条件
                        update_params = processed_row[1:] + [security_code]
                        cursor.execute(update_query, update_params)
                        update_count += 1
                        print(f"更新证券代码 {security_code} 的数据由{db_discount_rate}更改为{stock_discount_rate}")
                    else:
                        # 数据一致，跳过
                        skip_count += 1
                else:
                    # 数据不存在，执行插入
                    cursor.execute(insert_query, processed_row)
                    insert_count += 1
                    print(f"插入新证券代码 {security_code} 的数据")

            # 处理删除
            for code in codes_to_delete:
                cursor.execute(delete_query, (code,))
                delete_count += 1
                print(f"删除证券代码 {code} 的数据")

            # 提交事务
            connection.commit()
            print(f"\n操作完成:")
            print(f"  插入 {insert_count} 条新记录")
            print(f"  更新 {update_count} 条已有记录")
            print(f"  删除 {delete_count} 条多余记录")
            print(f"  跳过 {skip_count} 条无变化记录")
            print(f"  总共处理 {insert_count + update_count + skip_count + delete_count} 条记录")

    except Error as e:
        print(f"数据库错误: {e}")
        if connection.is_connected():
            connection.rollback()
            print("已回滚事务")
    except Exception as e:
        print(f"处理文件时出错: {e}")
        if connection.is_connected():
            connection.rollback()
            print("已回滚事务")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL连接已关闭")

# 使用示例
if __name__ == "__main__":
    # 数据库连接参数
    host = "localhost"
    database = "public"
    user = "root"
    password = "root"

    txt_file_path = "C:\\Users\\16528\\Desktop\\new 1.txt"  # 替换为您的TXT文件路径

    if os.path.exists(txt_file_path):
        import_txt_to_securities_info(txt_file_path, host, database, user, password)
    else:
        print("TXT文件不存在，请检查路径是否正确")

检测到文件编码: utf-8
查询数据库中已有数据...
数据库中已有 4043 条记录
删除证券代码 300131 的数据

操作完成:
  插入 0 条新记录
  更新 0 条已有记录
  删除 1 条多余记录
  跳过 4042 条无变化记录
  总共处理 4043 条记录
MySQL连接已关闭
